# ADP AMBER

![ADP!](ADP.png)

Here we will run through an example of using EMLE to perform an ML/MM simulation of Alanine-Dipeptide in water using AMBER.
We will be using default settings here, so using the ANI-2x non-reactive MLP and the generic emle embedding model.

The main differences between this and a normal submission script for an AMBER simulation are that you must launch an emle-server before the job, then stop the server after the job. Each step of the simulation, sander will send the information of the ML region to the server which predicts what the in vacuo energies and embedding energies and send it back. 

You can view the sander input/output files on the left, but they are exactly as you would expect for a normal sander simulation. Another file, "server_log.txt", is created which records the outputs from the server each step to say whether the calculation was successful or not.

In [ ]:
%%bash

cd ADP_AMBER
# Specify names in topology and coordinate files.
PARM=adp.parm7
CRD=adp.rst7

# Remove and re-create the output directory.
rm -rf output
mkdir output

# Switch to the output directory.
cd output

# Launch the emle-server in the background. (Sander will connect to this via ORCA.)
emle-server > server_log.txt 2>&1 &

# Launch sander.
sander -O -i ../emle.in -o emle.out -p ../adp.parm7 -c ../adp.rst7 -r emle.ncrst -x emle.nc > store.txt 2>&1

# Stop any running emle-server processes.
emle-stop

cd ../..

# ADP OpenMM

Now we will perform the same simulation but using OpenMM, where an interface to EMLE has been created using the Sire molecular simulation framework.

Here, we load the system first.

In [ ]:
import math
import os
import sys

import numpy as np
import scipy.io
import matplotlib.pyplot as plt

import openmm
import openmm.unit as unit
from openmm.app import *
from openmm import CustomBondForce, CustomCVForce

from emle.calculator import EMLECalculator

import sire as sr

# Load the ADP system for Sire
mols = sr.load_test_files("ala.crd", "ala.top")
# Load the topology for OpenMM too
prm = AmberPrmtopFile(f"ala.top")


# Creating the calculator
Next we create the calculator to perform the ML calculation. Whilst the AMBER implementation requires an external server, this is not the case for OpenMM. Instead, we define an EMLE calculator to use. Again, we use the defaults (ANI-2x and generic EMLE), and tell it to run on CPU (GPU is available but unnecessary for such a small system). 

To define the full system for simulation, we use "sr.qm.emle".
The first input, "mols", tells it all the whole system. Next, we give it the ML region ("mols[0]" which means just the ADP) and lastly we give it the calculator to use to predict the ML region energy.

In [ ]:
calculator = EMLECalculator(device="cpu")
qm_mols, engine = sr.qm.emle(mols, mols[0], calculator)

# Running the simulation

First we have to create the dynamics object. Here we use 1fs timestep with no constraints (as would be expected for "standard" QM/MM simulations). We then create the OpenMM context, setup the output trajectory file and launch the simulation. 

We run the simulation for 500 cycles of 10 steps, meaning we run the simulation for 5000 steps and every cycle (10 steps) the simulation writes to the trajectory.

In [ ]:
d = qm_mols.dynamics(
    timestep="1fs",
    constraint="none",
    qm_engine=engine,
    platform="cpu",
)


In [ ]:
from copy import deepcopy
context = d.context()
omm_system = context.getSystem()
integrator = deepcopy(context.getIntegrator())
new_context = openmm.Context(omm_system, integrator, context.getPlatform())
new_context.setPositions(context.getState(getPositions=True).getPositions())

In [ ]:
# Production sampling.
file_handle = open(f"ADP_OpenMM/ADP_OpenMM.dcd", "bw")
dcd_file = DCDFile(file_handle, prm.topology, dt=integrator.getStepSize())
for x in range(500):
    integrator.step(10)
    state = new_context.getState(getPositions=True)
    positions = state.getPositions()
    dcd_file.writeModel(positions)
file_handle.close()

# Viewing the trajectory

Here we use NGLviewer to view the output trajectory. The first option here views just the ADP whilst the second loads the whole ADP and water system.

In [ ]:
traj = sr.load("ala.top", "ADP_OpenMM/ADP_OpenMM.dcd")
traj[0].trajectory().view()

In [ ]:
traj = sr.load("ala.top", "ADP_OpenMM/ADP_OpenMM.dcd")
traj.trajectory().view()

# ADP Error Analysis

Another feature of emle-engine is the capability to perform error analysis of the model compared to QM/MM electrostatic embedding. By using a set of single point calculations from a trajectory generated by EMLE, the error in the static and induction components of the model can be found. Here you can compute these values for a pre-generated trajectory.

First we use the "emle-analyze" command which creates the output 


In [ ]:
%%bash
emle-analyze --qm-xyz qm.xyz \
             --pc-xyz pc.xyz \
             --emle-model emle_qm7_new_ivm0.1.mat \
             --orca-tarball generic.tar \
             --backend torchani analyze_generic.mat

In [ ]:
free_generic = scipy.io.loadmat('analyze_generic.mat', squeeze_me=True)


In [ ]:
def get_rmse(a, b):
    return np.sqrt(np.mean((a-b)**2))

def get_static_rmse(data):
    return get_rmse(data['E_static_qm'], data['E_static_emle'])

def get_induced_rmse(data):
    return get_rmse(data['E_induced_qm'], data['E_induced_emle'])

def get_total_rmse(data):
    return get_rmse(data['E_static_qm'] + data['E_induced_qm'], data['E_static_emle'] + data['E_induced_emle'])

def get_errors(data):
    static = get_static_rmse(data)
    induced = get_induced_rmse(data)
    total = get_total_rmse(data)
    print(f'static: {static:5.3f} induced: {induced:5.3f} total: {total:5.3f} ')

In [ ]:
get_errors(free_generic)


# ADP dihedral angles

Here you can see how to perform a simulation of the ADP with a biasing potential on the dihedral angles (for example for umbrella sampling)

In [ ]:
import math
import os
import sys

import numpy as np
import scipy.io
import matplotlib.pyplot as plt

import openmm
import openmm.unit as unit
from openmm.app import *
from openmm import CustomBondForce, CustomCVForce

from emle.calculator import EMLECalculator

import sire as sr

# Load the ADP system for Sire
mols = sr.load_test_files("ala.crd", "ala.top")
# Load the topology for OpenMM too
prm = AmberPrmtopFile(f"ala.top")

# Umbrella force constants for psi and phi.
k_psi = 100
k_phi = 100

# Add harmonic biasing potentials on two dihedrals of dialanine (psi, phi)
# in the OpenMM system for dihedral psi.
bias_torsion_psi = openmm.CustomTorsionForce(
    "0.5*k_psi*dtheta^2; dtheta = min(tmp, 2*pi-tmp); tmp = abs(theta - psi)"
)
bias_torsion_psi.addGlobalParameter("pi", math.pi)
bias_torsion_psi.addGlobalParameter("k_psi", 1.0)
bias_torsion_psi.addGlobalParameter("psi", 0.0)
# 4, 6, 8, 14 are indices of the atoms of the torsion psi.
bias_torsion_psi.addTorsion(4, 6, 8, 14)

# For dihedral phi.
bias_torsion_phi = openmm.CustomTorsionForce(
    "0.5*k_phi*dtheta^2; dtheta = min(tmp, 2*pi-tmp); tmp = abs(theta - phi)"
)
bias_torsion_phi.addGlobalParameter("pi", math.pi)
bias_torsion_phi.addGlobalParameter("k_phi", 1.0)
bias_torsion_phi.addGlobalParameter("phi", 0.0)
# 6, 8, 14, 16 are indices of the atoms of the torsion phi.
bias_torsion_phi.addTorsion(6, 8, 14, 16)

calculator = EMLECalculator(device="cpu")
qm_mols, engine = sr.qm.emle(mols, mols[0], calculator)